In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import ast
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


ModuleNotFoundError: No module named 'pandas'

In [3]:
papers = pd.read_csv('/kaggle/input/paper-prophet-challenge/trainingDATA.csv')
papers.head()

,ID,title,summary,year_time,categories
0,2301.02657v1,TarViS: A Unified Approach for Target-based Vi...,The general domain of video segmentation is cu...,2023-01-06 18:59:52+00:00,cs.CV cs.AI cs.LG
1,2301.02642v1,Triple-stream Deep Metric Learning of Great Ap...,We propose the first metric learning system fo...,2023-01-06 18:36:04+00:00,cs.CV cs.AI cs.LG
2,2301.02610v1,Feedback-Gated Rectified Linear Units,Feedback connections play a prominent role in ...,2023-01-06 17:14:11+00:00,cs.NE cs.AI
3,2301.02593v1,Multi-Agent Reinforcement Learning for Fast-Ti...,To integrate high amounts of renewable energy ...,2023-01-06 16:41:51+00:00,cs.MA cs.AI cs.LG cs.SY
4,2301.02561v1,Multi-Vehicle Trajectory Prediction at Interse...,Traditional approaches to prediction of future...,2023-01-06 15:13:23+00:00,cs.RO cs.AI


In [4]:
print(papers.shape)

(190000, 5)


In [5]:
papers = papers.drop(['year_time'], axis=1)

In [6]:
papers

,ID,title,summary,categories
0,2301.02657v1,TarViS: A Unified Approach for Target-based Vi...,The general domain of video segmentation is cu...,cs.CV cs.AI cs.LG
1,2301.02642v1,Triple-stream Deep Metric Learning of Great Ap...,We propose the first metric learning system fo...,cs.CV cs.AI cs.LG
2,2301.02610v1,Feedback-Gated Rectified Linear Units,Feedback connections play a prominent role in ...,cs.NE cs.AI
3,2301.02593v1,Multi-Agent Reinforcement Learning for Fast-Ti...,To integrate high amounts of renewable energy ...,cs.MA cs.AI cs.LG cs.SY
4,2301.02561v1,Multi-Vehicle Trajectory Prediction at Interse...,Traditional approaches to prediction of future...,cs.RO cs.AI
...,...,...,...,...
189995,0605045v1,On Orthogonalities in Matrices,In this paper we have discussed different poss...,cs.DM
189996,0605049v1,On fractionally linear functions over a finite...,"Abstrct: In this note, by considering fraction...",cs.DM
189997,0606087v3,Violator Spaces: Structure and Algorithms,Sharir and Welzl introduced an abstract framew...,cs.DM
189998,0607008v1,3-facial colouring of plane graphs,A plane graph is l-facially k-colourable if it...,cs.DM


In [7]:
df = papers[papers['categories'].notnull()]

In [8]:
categories = []
for x in df['categories']:
    y = x.split()
    for item in y:
        categories.append(item)

In [9]:
categories = list(dict.fromkeys(categories))

In [10]:
df2 = pd.DataFrame(columns=categories)

In [11]:
for x, idx in zip(df['categories'], range(0, df.shape[0])):
    y = x.split()
    row = []
    for item in df2.columns:
        if item in y:
            row.append(1)
        else:
            row.append(0)
    df2.loc[idx] = row
    if(idx%5000==0):
        print(idx)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000


In [19]:
df2

,cs.CV,cs.AI,cs.LG,cs.NE,cs.MA,cs.SY,cs.RO,cs.CL,cs.HC,cs.CC,...,cs.CE,cs.NA,cs.DL,cs.OH,cs.DM,cs.MS,cs.GL,cs.OS,"cs.CL,",cs.cc
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
186996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
186997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
186998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [26]:
df.shape, df2.shape

((187000, 4), (187000, 42))

In [46]:
df3 = pd.concat([df, df2], axis=1).reindex(df2.index)

In [48]:
df3 = df3.drop(['categories'], axis=1)

In [49]:
df3

,ID,title,summary,cs.CV,cs.AI,cs.LG,cs.NE,cs.MA,cs.SY,cs.RO,...,cs.CE,cs.NA,cs.DL,cs.OH,cs.DM,cs.MS,cs.GL,cs.OS,"cs.CL,",cs.cc
0,2301.02657v1,TarViS: A Unified Approach for Target-based Vi...,The general domain of video segmentation is cu...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2301.02642v1,Triple-stream Deep Metric Learning of Great Ap...,We propose the first metric learning system fo...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2301.02610v1,Feedback-Gated Rectified Linear Units,Feedback connections play a prominent role in ...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2301.02593v1,Multi-Agent Reinforcement Learning for Fast-Ti...,To integrate high amounts of renewable energy ...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2301.02561v1,Multi-Vehicle Trajectory Prediction at Interse...,Traditional approaches to prediction of future...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186995,0605131v2,Notes on Geometric Measure Theory Applications...,Regularization functionals that lower level se...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
186996,0609010v1,An effective edge--directed frequency filter f...,Raster images can have a range of various dist...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
186997,0609100v1,Total Variation Minimization and Graph Cuts fo...,"In this paper, we are interested in the applic...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
186998,0609164v1,Conditional Expressions for Blind Deconvolutio...,We present conditional expression (CE) for fin...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [50]:
bar_plot = pd.DataFrame()
bar_plot['cat'] = df3.columns[3:]
bar_plot['count'] = df3.iloc[:,3:].sum().values
bar_plot.sort_values(['count'], inplace=True, ascending=False)
bar_plot.reset_index(inplace=True, drop=True)
bar_plot.head()

,cat,count
0,cs.LG,39475.0
1,cs.AI,25004.0
2,cs.CR,14258.0
3,cs.CV,13222.0
4,cs.DC,12610.0


In [58]:
def decontract(sentence):
    # specific
    sentence = re.sub(r"won't", "will not", str(sentence))
    sentence = re.sub(r"can\'t", "can not", sentence)

    # general
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    return sentence

def cleanPunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', '', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub("", sentence)

In [52]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

re_stop_words = re.compile(r"\b(" + "|".join(stopwords) + ")\\W", re.I)

In [53]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

str

In [59]:
df3['summary'] = df3['summary'].str.lower()
df3['summary'] = df3['summary'].apply(decontract)
df3['summary'] = df3['summary'].apply(cleanPunc)
df3['summary'] = df3['summary'].apply(keepAlpha)
df3['summary'] = df3['summary'].apply(removeStopWords)
df3['summary'] = df3['summary'].apply(stemming)

In [60]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(df3.columns[3:])
y

array([[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        1],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1],
       [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
        1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
        1],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1],
       [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        1],
       [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0

In [159]:
X_train, X_test, y_train, y_test = train_test_split(df3['summary'], 
                                                    df3[df3.columns[3:]], 
                                                    test_size=0.3, 
                                                    shuffle=True)
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier


LR_pipeline = Pipeline([('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1))])
accuracy = 0
for category in df3.columns[3:]:
    print('**Processing {} titles...**'.format(category))
    
    # Training logistic regression model on train data
    LR_pipeline.fit(X_train, y_train[category])
    
    # calculating test accuracy
    prediction = LR_pipeline.predict(X_test)
    accuracy = accuracy + accuracy_score(y_test[category], prediction)
    print('AUC ROC score is {}'.format(roc_auc_score(y_test[category],prediction)))
    print("\n")

print('Test averaged AUC ROC is {}'.format(accuracy/len(df3.columns[3:])))

**Processing cs.CV titles...**
AUC ROC score is 0.6603697104408577


**Processing cs.AI titles...**
AUC ROC score is 0.5919674395726092


**Processing cs.LG titles...**
AUC ROC score is 0.7080311629567112


**Processing cs.NE titles...**
AUC ROC score is 0.7276377618189401


**Processing cs.MA titles...**
AUC ROC score is 0.5608238054430982


**Processing cs.SY titles...**
AUC ROC score is 0.5200871871261328


**Processing cs.RO titles...**
AUC ROC score is 0.5727731814699742


**Processing cs.CL titles...**
AUC ROC score is 0.6541665469795765


**Processing cs.HC titles...**
AUC ROC score is 0.7565443416170223


**Processing cs.CC titles...**
AUC ROC score is 0.5830753308172537


**Processing cs.GT titles...**
AUC ROC score is 0.8040607978081477


**Processing cs.GR titles...**
AUC ROC score is 0.5324120038275088


**Processing cs.LO titles...**


In [104]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(df3['summary'])
sequences = tokenizer.texts_to_sequences(df3['summary'])
x = pad_sequences(sequences, maxlen=200)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    df3[df3.columns[3:]], 
                                                    test_size=0.3)

In [69]:
num_classes = y_train.shape[1]
max_words = len(tokenizer.word_index) + 1
maxlen = 200

In [70]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [73]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
import tensorflow as tf
filter_length = 300

model = Sequential()
model.add(Embedding(max_words, 20, input_length=maxlen))
#model.add(Dropout(0.5))
model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])

callbacks = [
    ReduceLROnPlateau(),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=32,
                    validation_split=0.3,
                    callbacks=callbacks)

Epoch 1/25
2864/2864 [==============================] - 142s 49ms/step - loss: 0.1372 - auc_1: 0.8371 - val_loss: 0.1131 - val_auc_1: 0.8998 - lr: 0.0010
Epoch 2/25
2864/2864 [==============================] - 140s 49ms/step - loss: 0.1079 - auc_1: 0.9122 - val_loss: 0.1060 - val_auc_1: 0.9158 - lr: 0.0010
Epoch 3/25
2864/2864 [==============================] - 135s 47ms/step - loss: 0.1017 - auc_1: 0.9249 - val_loss: 0.1045 - val_auc_1: 0.9186 - lr: 0.0010
Epoch 4/25
2864/2864 [==============================] - 135s 47ms/step - loss: 0.0980 - auc_1: 0.9318 - val_loss: 0.1039 - val_auc_1: 0.9194 - lr: 0.0010
Epoch 5/25
2864/2864 [==============================] - 136s 47ms/step - loss: 0.0952 - auc_1: 0.9364 - val_loss: 0.1033 - val_auc_1: 0.9204 - lr: 0.0010
Epoch 6/25
2864/2864 [==============================] - 135s 47ms/step - loss: 0.0929 - auc_1: 0.9402 - val_loss: 0.1040 - val_auc_1: 0.9194 - lr: 0.0010
Epoch 7/25
2864/2864 [==============================] - 136s 47ms/step - los

In [74]:
test = pd.read_csv('/kaggle/input/paper-prophet-challenge/testingDATA .csv')

In [75]:
test.shape

(10094, 4)

In [76]:
test = test.drop(['year_time'], axis=1)

In [77]:
test['summary'] = test['summary'].str.lower()
test['summary'] = test['summary'].apply(decontract)
test['summary'] = test['summary'].apply(cleanPunc)
test['summary'] = test['summary'].apply(keepAlpha)
test['summary'] = test['summary'].apply(removeStopWords)
test['summary'] = test['summary'].apply(stemming)

In [150]:
tokenizer.fit_on_texts(test['summary'])
sequences = tokenizer.texts_to_sequences(test['summary'])
x = pad_sequences(sequences, maxlen=200)

In [81]:
X_train.shape

(130900, 200)

In [151]:
preds = model.predict(x)

316/316 [==============================] - 2s 6ms/step


In [ ]:
predclass=[]
for item in preds:
    predclass.append(np.argmax(item))
predclass

In [96]:
cnn_model = model
metrics = cnn_model.evaluate(X_test, y_test)
print("{}: {}".format(model.metrics_names[1], metrics[1]))

1754/1754 [==============================] - 10s 6ms/step - loss: 0.1162 - auc_1: 0.9005
auc_1: 0.9004915952682495


In [ ]:
classes = []
for item in predclass:
    t = str(categories[item])
    classes.append(t)
classes

In [156]:
submission = pd.DataFrame({
    'ID': test['ID'],
    'categories': classes
})

In [157]:
submission

,ID,categories
0,1202.6126v1,cs.SY
1,1111.5172v1,cs.DC
2,0602050v1,cs.LG
3,1107.4429v1,cs.LO
4,1609.09747v2,cs.LG
...,...,...
10089,0712.0871v1,cs.CR
10090,1205.3549v2,cs.LG
10091,1606.04646v1,cs.AI
10092,0310065v2,cs.CV


In [158]:
submission.to_csv('sub1.csv', index=False)